Lets try the most basic neural network, a MLP (Multi Layer Perceptron) or ANN. A MLP is a neural network that is feed-forward where each layer is connected to each subsequent layer (fully connected). 

In [ ]:
#todo insert images from mnist/tensorflow tutorial about MLP structure

In [1]:
import numpy as np # linear algebra
import os
import gc
import numpy
import pandas as pd
import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import h5py
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score

import cv2


Using TensorFlow backend.


In [2]:
train_path = 'input/train_jpg/'
#test_path = 'input/test_jpg/'
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/sample_submission.csv')

flatten = lambda x: [item for items in x for item in items]

labels = list(set(flatten([x.split(' ') for x in train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}
print(labels)
print(label_map)

['slash_burn', 'clear', 'blooming', 'primary', 'cloudy', 'conventional_mine', 'water', 'haze', 'cultivation', 'partly_cloudy', 'artisinal_mine', 'habitation', 'bare_ground', 'blow_down', 'agriculture', 'road', 'selective_logging']
{'selective_logging': 16, 'cultivation': 8, 'clear': 1, 'habitation': 11, 'conventional_mine': 5, 'cloudy': 4, 'primary': 3, 'water': 6, 'haze': 7, 'slash_burn': 0, 'partly_cloudy': 9, 'artisinal_mine': 10, 'blooming': 2, 'bare_ground': 12, 'blow_down': 13, 'agriculture': 14, 'road': 15}


In [3]:
def get_data(tag_df, path):
    x = []
    y = []
    for f, tags in tqdm(tag_df, miniters=1000):
#        try:
        img = cv2.imread('{}/{}.jpg'.format(path, f))
#        except Exception as  e:
#            print ('{}  error reading file {}'.format(e, f))
        targets = np.zeros(17)
        for t in tags.split(' '):
            targets[label_map[t]] = 1
        x.append(cv2.resize(img, (64, 64)))
        y.append(targets)
    return x, y

In [4]:
x_train, y_train = get_data(train.values, train_path)
y_train = np.array(y_train)
x_train = np.array(x_train)


100%|██████████| 40479/40479 [00:57<00:00, 707.39it/s]


In [5]:
x_train = x_train / 255.

In [6]:
from keras.preprocessing.image import ImageDataGenerator
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [14]:
from keras.optimizers import SGD
from keras.optimizers import Adam

from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(Flatten(input_shape = (64, 64, 3)))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


batch_size = 128

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_val, y_val))

from sklearn.metrics import fbeta_score

p_val = model.predict(x_val, batch_size=128)
print(y_val)
print(p_val)
print(fbeta_score(y_val, np.array(p_val) > 0.2, beta=2, average='samples'))
print(fbeta_score(y_val, np.array(p_val) > 0.2, beta=2, average='macro'))

Train on 32383 samples, validate on 8096 samples
Epoch 1/10
32383/32383 [==============================] - 7s - loss: 0.3704 - acc: 0.8957 - val_loss: 0.2427 - val_acc: 0.9095

/home/manava/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
